## 객체 지향으로 다시 만드는 학급 성적 평가 프로그램

### Stat 클래스
통계와 관련 있는 함수만 따로 분리해 클래스로 만든다

In [21]:
import math

class Stat:
    def average(self, scores):
        s = 0
        for score in scores:
            s += score
        return round(s/len(scores), 1)
    
    def variance(self, scores, avrg):
        s = 0
        for score in scores:
            s += (score - avrg) ** 2
        return round(s/len(scores), 1)
    
    def std_dev(self, variance):
        return round(math.sqrt(variance), 1)

### DataHandler 클래스
- Load Data from file
- 데이터를 연산하고 관리하는 클래스

In [28]:
from statistics import *
import openpyxl

class DataHandler:
    evaluator = Stat()
    
    @classmethod
    def get_data_from_excel(cls, filename):
        dic = {}
        wb = openpyxl.load_workbook(filename)
        ws = wb.active
        g = ws.rows
        
        for name, score in g:
            dic[name.value] = score.value
            
        return dic
    
    def __init__(self, filename, year_class):
        self.rawdata = DataHandler.get_data_from_excel(filename)
        self.year_class = year_class
        # 연산 값을 저장해 두는 저장소 
        self.cache = {}
        
    def get_scores(self):
        if 'scores' not in self.cache:
            self.cache['scores'] = list(self.rawdata.values())
        
        return self.cache.get('scores')
    
    def get_average(self):
        if 'average' not in self.cache:
            self.cache['average'] = self.evaluator.average(self.get_scores())
            
        return self.cache.get('average')
    
    def get_variance(self):
        if 'variance' not in self.cache:
            self.cache['variance'] = self.evaluator.variance(self.get_scores(), self.get_average())
            
        return self.cache.get('variance')
    
    def get_standard_deviation(self):
        if "standard_deviation" not in self.cache:
            self.cache["standard_deviation"] = self.evaluator.std_dev(
                self.get_variance())

        return self.cache.get("standard_deviation") 
    
    def evalute_class(self, total_avg, sd):
        avg = self.get_average()
        std_dev = self.get_standard_deviation()
        
        if avg < total_avg and std_dev > sd:
            print('성적이 너무 저조하고 학생들의 실력 차이가 너무 크다')
        elif avg > total_avg and std_dev > sd:
            print('[WARNING] 성적은 평균 이상이지만 학생들의 실력 차이가 크다.')
        elif avg < total_avg and std_dev < sd:
            print('학생들의 실력은 비슷하나 평균이 너무 낮음')
        elif avg > total_avg and std_dev < sd:
            print('애들 다 잘함. 걱정 없다!')
        
    def get_evaluation(self, total_avg, sd =20):
        print('='*50)
        print('{} 반 성적 분석 결과'.format(self.year_class))
        print(
        "{0} 반 \n평균 : {1} \n분산 : {2}, \n표준편차 : {3}"
            .format(self.year_class,
                   self.get_average(),
                   self.get_variance(),
                   self.get_standard_deviation()))
        print('='*50)
        print('{}반 종합 평가'.format(self.year_class))
        print('-'*50)
        self.evalute_class(total_avg, sd)
        

In [33]:
dh = DataHandler('class1.xlsx', '6-6')
dh.get_evaluation(50)

6-6 반 성적 분석 결과
6-6 반 
평균 : 51.5 
분산 : 1240.2, 
표준편차 : 35.2
6-6반 종합 평가
--------------------------------------------------
[WARNING] 성적은 평균 이상이지만 학생들의 실력 차이가 크다.
